In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType
from pyspark.sql import functions as sf

In [0]:
# Crafting Skill activities
# data is raw data from https://oldschool.runescape.wiki/w/Calculator:Crafting

df_crafting = spark.read.format("json") \
  .option("multiline", "true") \
  .load("/Volumes/runescape/00_landing/data_sources/skill_actions/crafting.json")

# select only columns that we care about and rename them to the correct column names
df_crafting = df_crafting.select(sf.col("level").alias("name"),
                                 sf.col("Input Cost").alias("input"),
                                 sf.col("# Needed").alias("xp"))

# need to split the input column
df = df_crafting.select('*', sf.split(df_crafting.input, '[\n]').alias("split_input"))



In [0]:
df.display()


In [0]:
df2 = df.select('name', sf.explode('split_input').alias("input"))
df2.display()

In [0]:
# split string by the multiplcation symbol '×'
df3 = df2.select('name', sf.split(df2.input, '[×]').alias("input"))
df3.display()

In [0]:
df4 = df3.select('name', sf.get(df3.input,0).alias("input_qty"), sf.get(df3.input,1).alias("input_name"))
df4.display()